# Capstone Applied Data Science Project
### Thomas KLEIN

This notebook will be used for the Capstone Data Science Project for the final course on Coursera of the IBM Data Science Professional Certificate.


In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto - Part 1: Data Wrangling

In this part we first scrape the Wikipedia page containing the data about neighborhoods in Toronto (link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) to create a dataframe.

In [2]:
#We import requests library to ping website for html code
import requests

In [3]:
#We install the beautifulsoup4 package
!pip install beautifulsoup4

     |████████████████████████████████| 102kB 8.6MB/s ta 0:00:011


In [4]:
#We also install a HTML parser
!pip install lxml

     |████████████████████████████████| 5.8MB 28.7MB/s eta 0:00:01


In [5]:
#We obtain html code of a website using requests.get(url).text
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [6]:
#We begin reading the html of the website using a Beautiful Soup object and the Prettify() function
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url, 'lxml')
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":920980179,"wgRevisionId":920980179,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNames

In [7]:
#We look for the object 'wikitable sortable' corresponding to the table we want to scrape
My_table = soup.find('table', class_='wikitable sortable')
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [8]:
#The data we want to capture is contained within the signs 'th' and 'td'. We start by reading only this bit of the table.
data=My_table.find_all(['th','td'][1::1])

#We initialize our 3 lists for postcodes, boroughs and neighbourhoods
Postcodes=[]
Boroughs=[]
Neighborhoods=[]

#We create a while loop to look through the table and assign the text in the first cell to Postcodes, in the second to boroughs and in the third to neighborhoods.
#Then we iterate i to i+3 to look in the next line.
i=0
while i in range(0,len(My_table.find_all('td'))):
    Postcode=data[i].text
    Borough=data[i+1].text
    Neighborhood=data[i+2].text
    Neighborhood=Neighborhood.strip()  #we remove the \n at the end of each Neighborhood name
    Postcodes.append(Postcode)
    Boroughs.append(Borough)
    Neighborhoods.append(Neighborhood)
    i=i+3

In [9]:
#We create a Dataframe to import each of our 3 created list as each one our columns.
df=pd.DataFrame()
df['Postcode']=Postcodes
df['Borough']=Boroughs
df['Neighborhood']=Neighborhoods
df

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [10]:
#Cleaning the dataframe to drop rows with unassigned borough and replace not assigned neighbohoods by the boroughs:
df=df[df['Borough']!='Not assigned'] #drop rows where Borough is 'Not assigned'

#replace the cell in the Neighborhood column by the one in the borough column if the value is 'Not assigned'
df.loc[df['Neighborhood']=='Not assigned', 'Neighborhood']=df['Borough'] 
df=df.reset_index(drop=True) #reset the index
df.head(15)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [103]:
#Now we use the groupby method to group Neighborhood belonging to the same Postcode.
index_cols=df.columns.tolist()
index_cols.remove('Neighborhood')
grouped_df = df.groupby(index_cols)['Neighborhood'].apply(list)
final_df=pd.DataFrame(grouped_df).reset_index()
#finally we transform the list of Neighborhoods in each Postcode into jjust a string of neighborhoods
final_df['Neighborhood']=[','.join(map(str, l)) for l in final_df['Neighborhood']]
final_df.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [104]:
print('The shape of the final prepared Dataframe is {} lines, {} columns'.format(final_df.shape[0],final_df.shape[1]))

The shape of the final prepared Dataframe is 103 lines, 3 columns


### End of part 1 - 1st submission of the notebook

## Segmenting and Clustering Neighborhoods in Toronto - Part 2: Importing the coordinates

In [105]:
#We import the csv file containing the coordinates (I tested the geocoder package but it didn't work properly)
url='http://cocl.us/Geospatial_data'
coordinates=pd.read_csv(url)
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [106]:
#We then merge the coordinates dataframe we created with the dataframe obtained at the end of part one.
df_merged=final_df.join(coordinates.set_index('Postal Code'), on='Postcode')
df_merged.head(10)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


### End of part 2 - 2nd submission of the notebook

## Segmenting and Clustering Neighborhoods in Toronto - Part 3: Clustering and exploring the neighborhoods

In [17]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.6.16-py36_1 conda-forge --> 2019.9.11-py36_0 conda-forge


certifi-2019.9.11    | 147 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [107]:
import folium

#We initialize the Folium map with Toronto's general coordinates
toronto_lat=43.651070
toronto_long=-79.347015
map_toronto_area = folium.Map(location=[toronto_lat, toronto_long], zoom_start=10)

#We start adding to the map the centers of the different neighborhoods listed in the dataframe we created previously
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'],df_merged['Longitude'],df_merged['Borough'], df_merged['Neighborhood']):
    label='{}{}'.format(neighborhood, borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
        ).add_to(map_toronto_area)

In [108]:
map_toronto_area

In [109]:
#As sugggested for the exercise we will only keep the data regarding Toronto's Boroughs so we decide to filter on the 4 Boroughs containing Toronto in their name
Toronto_Boroughs=['Central Toronto','Downtown Toronto','East Toronto','West Toronto']
Toronto_data = df_merged[df_merged['Borough'].isin(Toronto_Boroughs)].reset_index(drop=True)
Toronto_data.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [110]:
#We do the same as we did for the map of Toronto area but with a closer zoom on Toronto city only.
#We initialize the Folium map with Toronto's general coordinates
toronto_lat=43.651070
toronto_long=-79.347015
map_toronto = folium.Map(location=[toronto_lat, toronto_long], zoom_start=12)

#We start adding to the map the centers of the different neighborhoods listed in the dataframe we created previously
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'],Toronto_data['Longitude'],Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label='{}{}'.format(neighborhood, borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
        ).add_to(map_toronto)

#We display the map of Toronto city only
map_toronto

In [111]:
#Let's enter our Foursquare credentials and version
CLIENT_ID = 'SJPXHMOWC4S5KKV3GBLWIPSIGTY3R3QMBNOPJMU3SAY2Y1UB' # my Foursquare ID
CLIENT_SECRET = 'BC2OKDYBRT0Y2U43GFZNOJ3A4UKQH4P4CYPXYIAPNF4PVMSZ' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [112]:
#we use the get_category_type() function we defined in the labs to obtain the category of the venues
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [113]:
#We define the function to extract the venues nearby every borough
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [114]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )


The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [115]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1715, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [116]:
#We verify how many venues we have in each borough
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton,Exhibition Place,Parkdale Village",27,27,27,27,27,27
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",43,43,43,43,43,43
Central Bay Street,86,86,86,86,86,86
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [117]:
#We check how many unique categories of venues we have
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


#### Analysis of the neighborhoods

##### One Hot encoding

In [118]:
#one hot encoding
Toronto_onehot=pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood']=Toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns=[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[:-1])
Toronto_onehot=Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
#We group by neighborhood and average the number of venues
Toronto_grouped=Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head(10)

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.0,0.0,0.0,0.0,0.020000,0.000000,0.00,0.010000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.017857,0.000000,0.00,0.000000,0.000000,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.074074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00
6,Central Bay Street,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,...,0.0,0.0,0.0,0.0,0.011628,0.000000,0.00,0.011628,0.000000,0.00
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.060000,0.000000,0.04,0.010000,0.000000,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00
9,Church and Wellesley,0.022989,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,...,0.0,0.0,0.0,0.0,0.000000,0.011494,0.00,0.000000,0.011494,0.00


In [124]:
#We create a function to to sort values in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [155]:
#We create a new DataFrame to display the top 10 venues in each Neighborhood
num_top_venues=10
indicators=['st','nd','rd'] #endings for 1st, 2nd and 3rd

# create columns according to number of top venues
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{}Most common venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most common venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood']=Toronto_grouped['Neighborhood']
for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:]=return_most_common_venues(Toronto_grouped.iloc[ind,:], num_top_venues)
    
neighborhoods_venues_sorted.head(10)

,Neighborhood,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Thai Restaurant,American Restaurant,Steakhouse,Sushi Restaurant,Restaurant,Asian Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Beer Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Indian Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Café,Yoga Studio,Bakery,Breakfast Spot,Coffee Shop,Italian Restaurant,Nightclub,Convenience Store,Performing Arts Venue,Pet Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Coffee Shop,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boutique
5,"Cabbagetown,St. James Town",Bakery,Coffee Shop,Café,Pizza Place,Restaurant,Pub,Italian Restaurant,Gift Shop,Diner,Bank
6,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Ice Cream Shop,Burger Joint,Fried Chicken Joint,Gym / Fitness Center,Salad Place,Japanese Restaurant
7,"Chinatown,Grange Park,Kensington Market",Vegetarian / Vegan Restaurant,Café,Chinese Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Dumpling Restaurant,Coffee Shop,Bakery,Cocktail Bar
8,Christie,Café,Grocery Store,Park,Convenience Store,Nightclub,Diner,Baby Store,Italian Restaurant,Restaurant,Athletics & Sports
9,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Men's Store,Café,Burger Joint,Bubble Tea Shop


##### k Means clustering

In [156]:
from sklearn.cluster import KMeans
# set number of clusters
k_clusters=6
Toronto_grouped_clustering=Toronto_grouped.drop('Neighborhood',1)

# run k-means clustering
Kmeans=KMeans(n_clusters=k_clusters, random_state=0).fit(Toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster labels', Kmeans.labels_)
Toronto_merged=Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged=Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster labels,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Other Great Outdoors,Pub,Health Food Store,Trail,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Women's Store
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,5,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Pizza Place,Bookstore,Brewery,Bubble Tea Shop,Café
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,5,Brewery,Steakhouse,Sushi Restaurant,Italian Restaurant,Sandwich Place,Pub,Ice Cream Shop,Movie Theater,Burrito Place,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,5,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Brewery,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,5,Pizza Place,Breakfast Spot,Dance Studio,Food & Drink Shop,Clothing Store,Hotel,Sandwich Place,Park,Convenience Store,Gym
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,5,Coffee Shop,Sporting Goods Shop,Yoga Studio,Bagel Shop,Gym / Fitness Center,Clothing Store,Chinese Restaurant,Dessert Shop,Rental Car Location,Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,5,Dessert Shop,Sandwich Place,Coffee Shop,Sushi Restaurant,Italian Restaurant,Pizza Place,Café,Gym,Toy / Game Store,Farmers Market
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,0,Playground,Restaurant,Gym,Tennis Court,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,5,Coffee Shop,Pub,American Restaurant,Liquor Store,Light Rail Station,Sports Bar,Restaurant,Bagel Shop,Supermarket,Sushi Restaurant


##### Mapping clusters

In [158]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[toronto_lat, toronto_long], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

After experimenting with various numbers of clusters from 3 to 10, it seems like 6 was providing the most satisfying distribution of neighborhoods in clusters so as to not have one dominant cluster with almost all neighborhoods and then only 1-Neighborhood clusters.

##### Let's examine cluster 0

In [226]:
Cluster_0=Toronto_merged.loc[Toronto_merged['Cluster labels']==0,Toronto_merged.columns[[1]+list(range(5,Toronto_merged.shape[1]))]]
Cluster_0

,Borough,Cluster labels,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
8,Central Toronto,0,Playground,Restaurant,Gym,Tennis Court,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


In [228]:
print('Cluster {} is displayed in {} in the map above and contains {} cluster.'.format('0','red',Cluster_0.shape[0]))

Cluster 0 is displayed in red in the map above and contains 1 cluster.


##### Let's examine cluster 1

In [223]:
Cluster_1=Toronto_merged.loc[Toronto_merged['Cluster labels']==1,Toronto_merged.columns[[1]+list(range(5,Toronto_merged.shape[1]))]]
Cluster_1

,Borough,Cluster labels,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
25,Downtown Toronto,1,Café,Bakery,Restaurant,Japanese Restaurant,Bar,Bookstore,Sandwich Place,Flower Shop,Beer Bar,Beer Store
30,Downtown Toronto,1,Café,Grocery Store,Park,Convenience Store,Nightclub,Diner,Baby Store,Italian Restaurant,Restaurant,Athletics & Sports
31,West Toronto,1,Bakery,Supermarket,Pharmacy,Café,Bank,Bar,Music Venue,Middle Eastern Restaurant,Art Gallery,Park
33,West Toronto,1,Café,Yoga Studio,Bakery,Breakfast Spot,Coffee Shop,Italian Restaurant,Nightclub,Convenience Store,Performing Arts Venue,Pet Store
34,West Toronto,1,Café,Mexican Restaurant,Bar,Thai Restaurant,Flea Market,Furniture / Home Store,Music Venue,Diner,Discount Store,Cajun / Creole Restaurant


In [225]:
print('Cluster {} is displayed in {} in the map above and contains {} clusters.'.format('1','purple',Cluster_1.shape[0]))

Cluster 1 is displayed in purple in the map above and contains 5 clusters.


##### Let's examine cluster 2

In [220]:
Cluster_2=Toronto_merged.loc[Toronto_merged['Cluster labels']==2,Toronto_merged.columns[[1]+list(range(5,Toronto_merged.shape[1]))]]
Cluster_2

,Borough,Cluster labels,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
4,Central Toronto,2,Park,Swim School,Bus Line,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
10,Downtown Toronto,2,Park,Playground,Trail,Building,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
23,Central Toronto,2,Bus Line,Jewelry Store,Park,Sushi Restaurant,Trail,Women's Store,Dumpling Restaurant,Doner Restaurant,Donut Shop,Electronics Store


In [222]:
print('Cluster {} is displayed in {} in the map above and contains {} clusters.'.format('2','blue',Cluster_2.shape[0]))

Cluster 2 is displayed in blue in the map above and contains 3 clusters.


##### Let's examine cluster 3

In [218]:
Cluster_3=Toronto_merged.loc[Toronto_merged['Cluster labels']==3,Toronto_merged.columns[[1]+list(range(5,Toronto_merged.shape[1]))]]
Cluster_3

,Borough,Cluster labels,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
22,Central Toronto,3,Home Service,Garden,Women's Store,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [219]:
print('Cluster {} is displayed in {} in the map above and contains {} cluster.'.format('3','blue-green',Cluster_3.shape[0]))

Cluster 3 is displayed in blue-green in the map above and contains 1 clusters.


##### Let's examine cluster 4

In [214]:
Cluster_4=Toronto_merged.loc[Toronto_merged['Cluster labels']==4,Toronto_merged.columns[[1]+list(range(5,Toronto_merged.shape[1]))]]
Cluster_4

,Borough,Cluster labels,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,East Toronto,4,Other Great Outdoors,Pub,Health Food Store,Trail,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Women's Store


In [217]:
print('Cluster {} is displayed in {} in the map above and contains {} cluster.'.format('4','green',Cluster_4.shape[0]))

Cluster 4 is displayed in green in the map above and contains 1 cluster.


##### Let's examine cluster 5

In [208]:
Cluster_5=Toronto_merged.loc[Toronto_merged['Cluster labels']==5,Toronto_merged.columns[[1]+list(range(5,Toronto_merged.shape[1]))]]
Cluster_5

,Borough,Cluster labels,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
1,East Toronto,5,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Pizza Place,Bookstore,Brewery,Bubble Tea Shop,Café
2,East Toronto,5,Brewery,Steakhouse,Sushi Restaurant,Italian Restaurant,Sandwich Place,Pub,Ice Cream Shop,Movie Theater,Burrito Place,Fish & Chips Shop
3,East Toronto,5,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Brewery,Seafood Restaurant,Sandwich Place
5,Central Toronto,5,Pizza Place,Breakfast Spot,Dance Studio,Food & Drink Shop,Clothing Store,Hotel,Sandwich Place,Park,Convenience Store,Gym
6,Central Toronto,5,Coffee Shop,Sporting Goods Shop,Yoga Studio,Bagel Shop,Gym / Fitness Center,Clothing Store,Chinese Restaurant,Dessert Shop,Rental Car Location,Restaurant
7,Central Toronto,5,Dessert Shop,Sandwich Place,Coffee Shop,Sushi Restaurant,Italian Restaurant,Pizza Place,Café,Gym,Toy / Game Store,Farmers Market
9,Central Toronto,5,Coffee Shop,Pub,American Restaurant,Liquor Store,Light Rail Station,Sports Bar,Restaurant,Bagel Shop,Supermarket,Sushi Restaurant
11,Downtown Toronto,5,Bakery,Coffee Shop,Café,Pizza Place,Restaurant,Pub,Italian Restaurant,Gift Shop,Diner,Bank
12,Downtown Toronto,5,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Men's Store,Café,Burger Joint,Bubble Tea Shop
13,Downtown Toronto,5,Coffee Shop,Bakery,Café,Pub,Park,Mexican Restaurant,Restaurant,Breakfast Spot,Theater,Playground


In [216]:
print('Cluster {} is displayed in {} in the map above and contains {} clusters.'.format('5','orange',Cluster_5.shape[0]))

Cluster 5 is displayed in orange in the map above and contains 27 clusters.


##### Conclusion

In [233]:
columns_conclusion=['Cluster label', 'Color', 'Number of clusters', 'Main feature of the cluster']
Conclusion_table=pd.DataFrame(columns=columns_conclusion)
Conclusion_table['Cluster label']=range(k_clusters)
Conclusion_table['Color']=['Red','Purple','Blue','Blue-green','Green','Orange']
Conclusion_table['Number of clusters']=[Cluster_0.shape[0],Cluster_1.shape[0],Cluster_2.shape[0],Cluster_3.shape[0],Cluster_4.shape[0],Cluster_5.shape[0]]
Conclusion_table['Main feature of the cluster']=['Playground and sports activities','Mostly Cafés', 'Parks', 'Stores', 'Outdoors', 'Coffee shops, cafés and restaurants']

In [234]:
Conclusion_table

,Cluster label,Color,Number of clusters,Main feature of the cluster
0,0,Red,1,Playground and sports activities
1,1,Purple,5,Mostly Cafés
2,2,Blue,3,Parks
3,3,Blue-green,1,Stores
4,4,Green,1,Outdoors
5,5,Orange,27,"Coffee shops, cafés and restaurants"


### End of part 3 - 3rd submission of the notebook